##Installing the libraries


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install python-dotenv

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.7 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.whl size=2207716 sha256=48844a1eeb4a202bcb53354a98a766747d623626118d8becb7116945f29923b5
  Stored in directory: /root/.cache/pip/wheels/8a/ae/ec/235a682e0041fbaeee389843670581ec6c66872db856dfa9a4
Successfully built hnswlib


In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


## Setting Openai Environment


In [ ]:
import os
import openai
import sys
sys.path.append('../..')

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = 'API Key Here'

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [ ]:
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # Splitting
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    # Embeddings
    embeddings = OpenAIEmbeddings()
    # creating vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # Retrieval
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa


## Creating an Interface using panel

In [ ]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = None  # Initialize with None
        self.qa = None

    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified:
            return pn.pane.Markdown("Load a file")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = "temp.pdf"  # Store the loaded file path
            self.qa = load_db("temp.pdf", "stuff", 4)  # Load the PDF
            self.clr_history()
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("No DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return


In [ ]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput(placeholder='Ask any Question…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.Image('./img/convchain.jpg')

tab1 = pn.Column(
    pn.pane.Markdown(width=600, style={"background-color": "#f7f7f7", "padding": "8px"}),
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2 = pn.Column(
    pn.pane.Markdown(width=600, style={"background-color": "#f7f7f7", "padding": "8px"}),
    pn.layout.Divider(),
    pn.Row(file_input, button_load, bound_button_load),
    pn.layout.Divider(),
    pn.Row(button_clearhistory, pn.pane.Markdown("Clear chat history. Can upload a new file.")),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
tab3 = pn.Column(
    pn.pane.Markdown(width=600, style={"background-color": "#f7f7f7", "padding": "8px"}),
    pn.layout.Divider(),
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4 = pn.Column(
    pn.pane.Markdown(width=600, style={"background-color": "#f7f7f7", "padding": "8px"}),
    pn.layout.Divider(),
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources),
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA ChatBot Using PDF File', style={"font-size": "16px"})),
    pn.Tabs(('Conversation', tab1), ('Upload File', tab2), ('Chat History', tab3), ('Database', tab4))
)
dashboard


<ipython-input-44-875cf0a34870>:15: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.pane.Markdown(width=600, style={"background-color": "#f7f7f7", "padding": "8px"}),
<ipython-input-44-875cf0a34870>:22: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.pane.Markdown(width=600, style={"background-color": "#f7f7f7", "padding": "8px"}),
<ipython-input-44-875cf0a34870>:31: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.pane.Markdown(width=600, style={"background-color": "#f7f7f7", "padding": "8px"}),
<ipython-input-44-875cf0a34870>:37: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.pane.Markdown(width=600, style={"background-color": "#f7f7f7", "padding": "8px"}),
<ipython-input-44-875cf0a34870>:44: PanelDeprecationWarning: 'style' is deprecated a

Column
    [0] Row
        [0] Markdown(str, styles={'font-size': '16px'})
    [1] Tabs
        [0] Column
            [0] Markdown(None, styles={'background-color': '#f7f...}, width=600)
            [1] Row
                [0] TextInput(placeholder='Ask any Question…')
            [2] Divider()
            [3] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [4] Divider()
        [1] Column
            [0] Markdown(None, styles={'background-color': '#f7f...}, width=600)
            [1] Divider()
            [2] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [3] Divider()
            [4] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [5] Divider()
            [6] Row
                [0] Image(str, width=400)
        [2] Column
            [0] Markdown(None, styles={'background-color': '#f7f...}, width=600)
            [1] Divider()
            [2] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [3] Divider()
        [3] Column
            [0] Markdown(None, styles={'background-color': '#f7f...}, width=600)
            [1] Divider()
            [2] ParamMethod(method, _pane=Column, defer_load=False)
            [3] Divider()
            [4] ParamMethod(method, _pane=Str, defer_load=False)

<ipython-input-42-709cd18cc971>:36: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
